#Census Tract Socio Demographic features of working population in New York City (Analysis  based on LEHD 2014 Origin-Destination Data)

The main purpose of this notebook, is to get the approximate statistics on certain demographic features of New York City workers by Census tracts, based on their residence location. 

LEHD dataset provides information about the number of workers comming from a given census block to each census block in New York City. Specifically, the datasets used in our analysis are: 'ny_od_main_JT00_2014.csv' (information about workers in New YorkThe  State with residence in New York State) and 'ny_od_aux_JT00_2014.csv' (information about workers in New York State with residence outside the State).

As a first step, an exploratory analysis is performed. The goal of this exploration is to find from which states and counties the working population is comming to work in NYC everyday. Using the geocode of this counties, socio-demographic features are downloaded from Social Explorer (Census Data) and then merged again with the LEHD data set.

Each pair of census blocks (residence CB and workplace CB) has the information about the number of workers. This data is aggregated by census tract and then a weighted sum is performed for each feature and each census block.

Let $\text{ w _ geo }_i$ be the geocode of the working place (where $i=1, ... ,m$), and $\text{ h _ geo }_j$ be the geocode of the residence place (where $j=1, ... ,n$), the LEHD data structure can be seen as follows:

$$\text{ w _ geo }_i \left\{\begin{matrix}
\text{ h_ geo }_1 & n_{i,1} \\ 
\text{ h_ geo }_2 & n_{i,2}\\ 
... & ... \\ 
\text{ h_ geo }_m & n_{i,m}
\end{matrix}\right.$$ where $n_{i,j}$ is the number of workers commuting from $\text{ h_ geo }_j$ to $\text{ w _ geo }_i$


From Census Data, we get a collection of $k=1, ... , l$ socio-demographic features for each census tract $\text{ h_ geo }_j$. Lets name each of this features as: $Pc_k ( \text{ h_ geo }_j ) := Pc_{k,j}$, and $\bar{Pc_{k,j}}$ its normalized version.

The goal is to get an approximate of each of this features for each $\text{ w_ geo }_i$. Let $F_k (\text{ w_ geo }_i)$ be the approximate number of workers which workplace is in $\text{ w_ geo }_i$, in the $k$ socio-demographic cathegory. 

Then,  $$F_k (\text{ w_ geo }_i)  = \sum_{j} \overline{Pc_{kj}} * n_{i,j} $$ for each $\text{ w_ geo }_i$ and for each socio-demographic indicator $k$.

In [8]:
import numpy as np
import pandas as pd

There are two different Origin-Destination files 'main' and 'aux'. The first one contains h_geocodes inside NY state, while the 'aux' file contain resident's blocks outisde NY. First, we are going to combine both sets. 

In [9]:
LEHD_original=pd.read_csv('datasets/ny_od_main_JT00_2014.csv')

In [10]:
print LEHD_original.shape
LEHD_original.head()

(6989032, 13)


,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
0,360010001001004,360010137054014,1,0,0,1,0,0,1,0,1,0,20160219
1,360010001001004,360010138023000,1,0,0,1,0,0,1,0,1,0,20160219
2,360010001001004,360830522041091,1,0,1,0,0,0,1,0,1,0,20160219
3,360010001001004,360910627001014,1,0,1,0,0,0,1,0,1,0,20160219
4,360010001001005,360010001001023,1,0,1,0,0,1,0,0,1,0,20160219


In [11]:
LEHD_original2=LEHD_original[['w_geocode','h_geocode','S000']]

In [12]:
LEHD_aux=pd.read_csv('datasets/ny_od_aux_JT00_2014.csv')
LEHD_aux2=LEHD_aux[['w_geocode','h_geocode','S000']]
LEHD_aux2.head()

,w_geocode,h_geocode,S000
0,360010001001006,250039009005009,1
1,360010001001008,60590879011008,1
2,360010001001008,60590997023009,1
3,360010001001008,90012301002025,1
4,360010001001008,250039231002006,1


In [13]:
LEHD_complete=LEHD_original2.append(LEHD_aux2)
print len(LEHD_complete),len(LEHD_original2),len(LEHD_aux2)

7673034 6989032 684002


Now, the LEHD_complete dataset is going to be subseted for getting workplaces inside New York City

In [14]:
#SI 085 - 5
#QN 081 - 4
#BX 005 - 2
#BK 047 - 3
#MN 061 - 1

In [15]:
#LEHD_complete['h_state_code']=map(lambda x: str(x)[0:2],LEHD_complete.h_geocode)    #origin state code
#LEHD_complete['h_county_code']=map(lambda x: str(x)[2:5],LEHD_complete.h_geocode)  #origin county code
#LEHD_complete['h_ct_code']=map(lambda x: str(x)[5:11],LEHD_complete.h_geocode)     #origin census tract
LEHD_complete['w_county_code']=map(lambda x: str(x)[2:5],LEHD_complete.w_geocode)   #destination county code
#LEHD_complete['w_ct_code']=map(lambda x: str(x)[5:11],LEHD_complete.w_geocode)     #destination census tract

In [16]:
LEHD_complete.head()

,w_geocode,h_geocode,S000,w_county_code
0,360010001001004,360010137054014,1,001
1,360010001001004,360010138023000,1,001
2,360010001001004,360830522041091,1,001
3,360010001001004,360910627001014,1,001
4,360010001001005,360010001001023,1,001


In [17]:
counties_NYC=['085','081','005','047','061']
mask1=map(lambda x: x in counties_NYC,LEHD_complete.w_county_code) 
LEHD_NYC=LEHD_complete[mask1]
print len(LEHD_NYC), len(LEHD_complete)

3352852 7673034


The goal now is to know which origin census tracts are outside New York City, and where are they located. First, I am going to group by county of origin and count how many workers come from each county.

In [18]:
#SINCE there are states with state code less than 10, I will create filters for those.
def state_func(st):
    if len(str(st))==15:
        return str(st)[0:2]
    else:
        return str(st)[0:1]
def county_func(st):
    if len(str(st))==15:
        return str(st)[2:5]
    else:
        return str(st)[1:4]
def ct_func(st):
    if len(str(st))==15:
        return str(st)[5:11]
    else:
        return str(st)[4:10]

In [19]:
LEHD_complete['w_ct_code']=map(lambda x: str(x)[5:11],LEHD_complete.w_geocode)     #destination census tract
LEHD_complete['h_state_code']=map(lambda x: state_func(x),LEHD_complete.h_geocode)    #origin state code
LEHD_complete['h_county_code']=map(lambda x:  county_func(x),LEHD_complete.h_geocode)  #origin county code
LEHD_complete['h_ct_code']=map(lambda x: ct_func(x),LEHD_complete.h_geocode)     #origin census tract


In [20]:
#LEHD_complete['w_ct_code']=map(lambda x: str(x)[5:11],LEHD_complete.w_geocode)     #destination census tract
#LEHD_complete['h_state_code']=map(lambda x: str(x)[0:2],LEHD_complete.h_geocode)    #origin state code
#LEHD_complete['h_county_code']=map(lambda x: str(x)[2:5],LEHD_complete.h_geocode)  #origin county code
#LEHD_complete['h_ct_code']=map(lambda x: str(x)[5:11],LEHD_complete.h_geocode)     #origin census tract


In [21]:
LEHD_complete.head()

,w_geocode,h_geocode,S000,w_county_code,w_ct_code,h_state_code,h_county_code,h_ct_code
0,360010001001004,360010137054014,1,001,000100,36,001,013705
1,360010001001004,360010138023000,1,001,000100,36,001,013802
2,360010001001004,360830522041091,1,001,000100,36,083,052204
3,360010001001004,360910627001014,1,001,000100,36,091,062700
4,360010001001005,360010001001023,1,001,000100,36,001,000100


In [22]:
origin_by_county=LEHD_complete[['h_state_code','h_county_code','S000']]

In [23]:
origin_by_county['county_code']=origin_by_county.h_state_code+origin_by_county.h_county_code

C:\Users\Marco\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
or_by_county=origin_by_county.groupby('county_code').sum().reset_index()

In [25]:
def state_func(st):
    if len(str(st))==5:
        return str(st)[0:2]
    else:
        return str(st)[0:1]
def county_func(st):
    if len(str(st))==5:
        return str(st)[2:5]
    else:
        return str(st)[1:4]

In [26]:
or_by_county['state_number']=map(lambda x: state_func(x),or_by_county.county_code)  
or_by_county['county_number']=map(lambda x: county_func(x),or_by_county.county_code)  

In [27]:
or_by_state=or_by_county.groupby('state_number').sum().reset_index()

In [28]:
#or_by_state.sort(columns='S000',ascending=False)

A refference with the states and its number on census can be find at: http://www2.census.gov/geo/docs/reference/state.txt

In [29]:
#According to this calculations, the states with more workers commuting to NYC everyday are: 
#36	8097590 NY
#34	469113 NJ
#9	89699 CT
#42	69146 PA
#25	25410 MA

In [ ]:
or_by_county.head()

,county_code,S000,state_number,county_number
0,10001,474,10,001
1,10003,1748,10,003
2,10005,618,10,005
3,1001,2,1,001
4,1003,19,1,003


Now, let's answer the question: How many workers do we lose if we only consider those 5 states: NY,NJ,CT,PA,MA

In [ ]:
states_to_NYC=['36','34','9','42','25']
mask2=map(lambda x: x in states_to_NYC,LEHD_complete.h_state_code) 
LEHD_complete2=LEHD_complete[mask2]
print len(LEHD_complete2), len(LEHD_complete)

7603524 7673034


In [ ]:
((7673034.0-7603524.0)/7673034.0)*100

0.9058998044319887

That means that if we only consider Census Tract on those 5 states, we only lose 0.9% of the data

Now, let's generate a list of the counties that are going to be needed in our study

In [ ]:
#Saving the progress so far
LEHD_complete2.to_csv('LEHD_analysis_6_13.csv')

In [ ]:
dtype={'w_geocode': str, 'h_geocode': str, 'S000': np.int32, 'w_county_code': str,'w_ct_code':str, 
       'h_state_code': str, 'h_county_code': str, 'h_ct_code': str}

LEHD_complete2=pd.read_csv('LEHD_analysis_6_13.csv',dtype=dtype)

In [ ]:

LEHD_complete2.head()

,Unnamed: 0,w_geocode,h_geocode,S000,w_county_code,w_ct_code,h_state_code,h_county_code,h_ct_code
0,0,360010001001004,360010137054014,1,001,000100,36,001,013705
1,1,360010001001004,360010138023000,1,001,000100,36,001,013802
2,2,360010001001004,360830522041091,1,001,000100,36,083,052204
3,3,360010001001004,360910627001014,1,001,000100,36,091,062700
4,4,360010001001005,360010001001023,1,001,000100,36,001,000100


In [ ]:
origin_by_county=LEHD_complete2[['h_state_code','h_county_code','S000']]
origin_by_county['county_code']=origin_by_county.h_state_code+origin_by_county.h_county_code

C:\Users\Marco\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [ ]:
county_list={}
for state in np.unique(origin_by_county.h_state_code):
    origin_by_county_state=origin_by_county[origin_by_county.h_state_code==state]
    county_list[state]=np.unique(origin_by_county_state.h_county_code)

In [ ]:
county_list

{'25': array(['001', '003', '005', '007', '009', '011', '013', '015', '017',
        '019', '021', '023', '025', '027'], dtype=object),
 '34': array(['001', '003', '005', '007', '009', '011', '013', '015', '017',
        '019', '021', '023', '025', '027', '029', '031', '033', '035',
        '037', '039', '041'], dtype=object),
 '36': array(['001', '003', '005', '007', '009', '011', '013', '015', '017',
        '019', '021', '023', '025', '027', '029', '031', '033', '035',
        '037', '039', '041', '043', '045', '047', '049', '051', '053',
        '055', '057', '059', '061', '063', '065', '067', '069', '071',
        '073', '075', '077', '079', '081', '083', '085', '087', '089',
        '091', '093', '095', '097', '099', '101', '103', '105', '107',
        '109', '111', '113', '115', '117', '119', '121', '123'], dtype=object),
 '42': array(['001', '003', '005', '007', '009', '011', '013', '015', '017',
        '019', '021', '023', '025', '027', '029', '031', '033', '035',
        '03

New York Counties codes: http://library.columbia.edu/locations/dssc/data/nycounty_fips.html
New Jersey Counties codes: https://www.epa.gov/enviro/county-fips-code-listing-state-new-jersey
Connecticut Counties codes:

In [ ]:
np.unique(origin_by_county['county_code'])

array(['25001', '25003', '25005', '25007', '25009', '25011', '25013',
       '25015', '25017', '25019', '25021', '25023', '25025', '25027',
       '34001', '34003', '34005', '34007', '34009', '34011', '34013',
       '34015', '34017', '34019', '34021', '34023', '34025', '34027',
       '34029', '34031', '34033', '34035', '34037', '34039', '34041',
       '36001', '36003', '36005', '36007', '36009', '36011', '36013',
       '36015', '36017', '36019', '36021', '36023', '36025', '36027',
       '36029', '36031', '36033', '36035', '36037', '36039', '36041',
       '36043', '36045', '36047', '36049', '36051', '36053', '36055',
       '36057', '36059', '36061', '36063', '36065', '36067', '36069',
       '36071', '36073', '36075', '36077', '36079', '36081', '36083',
       '36085', '36087', '36089', '36091', '36093', '36095', '36097',
       '36099', '36101', '36103', '36105', '36107', '36109', '36111',
       '36113', '36115', '36117', '36119', '36121', '36123', '42001',
       '42003', '420

The next step is to group the w_geocode and h_geocode by census tracts

In [ ]:
LEHD_complete2['w_geo_ct']='36'+LEHD_complete2.w_county_code+LEHD_complete2.w_ct_code
LEHD_complete2['h_geo_ct']=LEHD_complete2.h_state_code+LEHD_complete2.h_county_code+LEHD_complete2.h_ct_code

In [ ]:
LEHD_complete2.head()

,Unnamed: 0,w_geocode,h_geocode,S000,w_county_code,w_ct_code,h_state_code,h_county_code,h_ct_code,w_geo_ct,h_geo_ct
0,0,360010001001004,360010137054014,1,001,000100,36,001,013705,36001000100,36001013705
1,1,360010001001004,360010138023000,1,001,000100,36,001,013802,36001000100,36001013802
2,2,360010001001004,360830522041091,1,001,000100,36,083,052204,36001000100,36083052204
3,3,360010001001004,360910627001014,1,001,000100,36,091,062700,36001000100,36091062700
4,4,360010001001005,360010001001023,1,001,000100,36,001,000100,36001000100,36001000100


In [ ]:
grouped_data=LEHD_complete2.groupby(['w_geo_ct','h_geo_ct']).sum().reset_index().drop('Unnamed: 0',axis=1)
grouped_data.head()

,w_geo_ct,h_geo_ct,S000
0,36001000100,25003900100,2
1,36001000100,25003900300,4
2,36001000100,25003900400,2
3,36001000100,25003900500,1
4,36001000100,25003900800,1


In [ ]:
#saving progress
grouped_data.to_csv('outputs/LEHD_analysis/LEHD_by_CT.csv')

Now, the census data will be imported

In [ ]:
grouped_data=pd.read_csv('outputs/LEHD_analysis/LEHD_by_CT.csv').drop('Unnamed: 0',axis=1)
grouped_data.head()

,w_geo_ct,h_geo_ct,S000
0,36001000100,25003900100,2
1,36001000100,25003900300,4
2,36001000100,25003900400,2
3,36001000100,25003900500,1
4,36001000100,25003900800,1


In [ ]:
#file with features for census tracts inside NYC
demographics_CT_NYC_residents=pd.read_csv('datasets/emographics_CT_NYC_residents.csv')
demographics_CT_outNY=pd.read_csv('datasets/demographics_CT_outNY.csv')

In [ ]:
demographics_CT_NYC_residents.drop('BoroCT2010',axis=1,inplace=True)

In [ ]:
demographics_CT_outNY.columns=demographics_CT_NYC_residents.columns

In [ ]:
demographics_all_CT=demographics_CT_NYC_residents.append(demographics_CT_outNY)
print len(demographics_all_CT),len(demographics_CT_NYC_residents),len(demographics_CT_outNY)

In [ ]:
#from the demographics file, I will take only those census tracts that appear on  the LEHD file
h_geo_ct_origins=pd.DataFrame(np.unique(grouped_data['h_geo_ct']),columns=['Geo_FIPS'])

In [ ]:
demographics_CT_origins=pd.merge(h_geo_ct_origins,demographics_all_CT,on='Geo_FIPS',how='inner')
print len(h_geo_ct_origins),len(demographics_CT_origins), np.float(len(h_geo_ct_origins))/np.float(len(demographics_CT_origins))

In [ ]:
#demographics_CT_origins must be normalized (transformed into a probabilities matrix)
demographics_CT_origins.to_csv('outputs/LEHD_analysis/demographics_CT_origins.csv')

In [ ]:
demographics_CT_origins_norm=pd.read_csv('outputs/LEHD_analysis/demographics_CT_origins.csv')
demographics_CT_origins_norm.head()

In [ ]:
#demographics_CT_origins=demographics_CT_origins.rename(columns={'Geo_FIPS':'h_geo_ct'})
#pre_working_demographics1=pd.merge(grouped_data,demographics_CT_origins,on='h_geo_ct',how='left')
#print len(grouped_data),len(pre_working_demographics1)

In [ ]:
#get a colummn of the number (c) of workers going from h_geo to w_geo divided by the total working population in w_geo. 
#then c will be multiplied by the demographic indicators and summed
df1=grouped_data.groupby('w_geo_ct').sum().reset_index().drop('h_geo_ct',axis=1).rename(columns={'S000':'total_workers_in_wgeo'})
df2=pd.merge(grouped_data,df1,on='w_geo_ct',how='inner')
#df2['coefficient_c']=df2['S000']/df2.total_workers_in_wgeo
#df2.drop(['S000','total_workers_in_wgeo'],axis=1, inplace=True)
df2.head()

In [ ]:
demographics_CT_origins_norm=demographics_CT_origins_norm.rename(columns={'Geo_FIPS':'h_geo_ct'})
test2=pd.merge(df2,demographics_CT_origins_norm,on='h_geo_ct',how='left')

In [ ]:
test_dict={}
result_dict={}
for w_geo in np.unique(test2.w_geo_ct):
    test_dict[w_geo]=test2[test2.w_geo_ct==w_geo].drop(['w_geo_ct','h_geo_ct'],axis=1).convert_objects(convert_numeric=True)
    A=test_dict[w_geo]
    B=A.drop(['S000','total_workers_in_wgeo'],axis=1)
    B=B.multiply(A.S000, axis=0)
    B['total workers']=A.S000
    result_dict[w_geo]=B.sum()

In [ ]:
result_dict[36047044600]

In [ ]:
Z=pd.DataFrame(result_dict)

In [ ]:
Z2=Z.transpose().reset_index().rename(columns={'index':'Geo_FIPS'})
Z2.to_csv('outputs/LEHD_analysis/demographics_CT_NYC_workers.csv')